# Carga de datos

Se utilizan los archivos Excel con los resultados oficiales de las elecciones
parlamentarias de 2021 en Chile, entregados por el
[Tricel](https://tricel.cl/resultados-de-elecciones/).

A cada candidato se le agrega el pacto al cual pertenece y el distrito por el
cual está compitiendo, guardando la información en `datos.csv`.

In [10]:
import numpy as np
import pandas as pd

In [11]:
# function that receives a candidate row, and returns the pact to which the
# candidate belongs (the first row above the candidate that doesn't have a 
# vote tally)
def get_pact(row, df):
    
    if np.isnan(row["votes"]):
        return None
    
    for i in range(row.name, -1, -1):
        
        if np.isnan(df.loc[i]["votes"]):
            return df.loc[i]["candidate"]
        
    return None

In [12]:
results = pd.DataFrame(
    np.empty((0, 5)),
    columns=["candidate", "pact", "party", "district", "votes"]
)

# for each district
for n in range(1, 29):
    
    # load excel
    district_df = pd.read_excel(
        f"datos/Distrito-{n}-DIPUTADOS-2021.xlsx",
        sheet_name="CANDIDATOS",
        header=5,
        names=["candidate", "party", "votes"],
        usecols="C:E"
        )
    # remove rows that tally votes for each pact
    district_df = district_df[
        ~(district_df["party"].isna())
        | (district_df["votes"].isna())
    ]

    # assign pact and district to each candidate
    district_df["pact"] = district_df.apply(
        get_pact, args=(district_df,), axis=1
    )
    district_df["district"] = n

    # remove pact rows and reorder columns
    district_df = district_df[~district_df["party"].isna()][
        ["candidate", "pact", "party", "district", "votes"]
    ]

    # concatenate to main dataframe
    results = pd.concat((results, district_df), axis=0)

# reset index, drop null candidates (edge case) and remove independents
results = results.reset_index().iloc[:, 1:]
results = results.drop(results[results["candidate"].isna()].index)
results = results.drop(results[results["party"] == "INDEPENDIENTES"].index)

# cast district number and vote tallies to integers
results[["district", "votes"]] = results[["district", "votes"]].astype(int)

In [13]:
# rename parties and pacts
results = results.replace({
    "pact": {
        "CHILE PODEMOS +": "ChP+",
        "PARTIDO DE LA GENTE": "PDG",
        "PARTIDO DE TRABAJADORES REVOLUCIONARIOS": "PTR",
        "NUEVO PACTO SOCIAL": "NPS",
        "DIGNIDAD AHORA": "DA",
        "FRENTE SOCIAL CRISTIANO": "FSC",
        "APRUEBO DIGNIDAD": "AD",
        "NUEVO TIEMPO": "NT",
        "INDEPENDIENTES UNIDOS": "IU",
        "PARTIDO PROGRESISTA DE CHILE": "PRO",
        "PARTIDO ECOLOGISTA VERDE": "PEV",
        "UNION PATRIOTICA": "UPA"
    },
    "party": {
        "UNION DEMOCRATA INDEPENDIENTE": "UDI",
        "EVOLUCION POLITICA": "EVO",
        "RENOVACION NACIONAL": "RN",
        "PARTIDO REGIONALISTA INDEPENDIENTE DEMOCRATA": "PRI",
        "PARTIDO DE LA GENTE": "PDG",
        "PARTIDO DE TRABAJADORES REVOLUCIONARIOS": "PTR",
        "PARTIDO LIBERAL DE CHILE": "PL",
        "PARTIDO SOCIALISTA DE CHILE": "PS",
        "PARTIDO HUMANISTA": "PH",
        "PARTIDO REPUBLICANO DE CHILE": "PLR",
        "FEDERACION REGIONALISTA VERDE SOCIAL": "FREVS",
        "COMUNES": "COM",
        "PARTIDO COMUNISTA DE CHILE": "PC",
        "NUEVO TIEMPO": "NT",
        "PARTIDO POR LA DEMOCRACIA": "PPD",
        "PARTIDO DEMOCRATA CRISTIANO": "PDC",
        "REVOLUCION DEMOCRATICA": "RD",
        "PARTIDO RADICAL DE CHILE": "PR",
        "IGUALDAD": "PI",
        "CENTRO UNIDO": "CU",
        "PARTIDO PROGRESISTA DE CHILE": "PRO",
        "PARTIDO ECOLOGISTA VERDE": "PEV",
        "CONVERGENCIA SOCIAL": "CS",
        "UNION PATRIOTICA": "UPA",
        "CIUDADANOS": "CIU",
        "PARTIDO CONSERVADOR CRISTIANO": "PCC",
        "PARTIDO NACIONAL CIUDADANO": "PNC"
    }
})

In [16]:
# most and least voted candidates
results.sort_values("votes", ascending=False)

,candidate,pact,party,district,votes
459,KAROL AIDA CARIOLA OLIVA,AD,PC,9,78837
643,PAMELA JILES MORENO,DA,PH,12,77593
533,GONZALO WINTER ETCHEBERRY,AD,CS,10,66794
753,MARISELA SANTIBAÑEZ NOVOA,AD,PC,14,55932
779,RAUL SOTO MARDONES,NPS,PPD,15,55346
...,...,...,...,...,...
522,JUAN EGOR PLAZA AGUILAR,DA,PI,10,269
298,PABLO VALENTIN RECABAL MATURANA,UPA,UPA,7,230
1137,TERESA MATILDE ALBERTINA ALMONACID ALMONACID,DA,PH,24,202
1223,AMERICO EUGENIO AEDO URRA,PDG,PDG,27,156


In [ ]:
# save data to csv file
results.to_csv("datos.csv")